In [175]:
import os
import cv2
import numpy as np
from PIL import Image as ImagePIL
from matplotlib import pyplot as plt

import sys
import os
from PIL import ImageDraw, ImageFont
from IPython.display import display
from paddleocr import PaddleOCR, draw_ocr
import pytesseract
import pandas as pd

In [176]:
def deskew(im, max_skew=10):
    height, width = im.shape

    # Create a grayscale image and denoise it
    #im_gs = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gs = im
    im_gs = cv2.fastNlMeansDenoising(im_gs, h=3)

    # Create an inverted B&W copy using Otsu (automatic) thresholding
    im_bw = cv2.threshold(im_gs, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Detect lines in this image. Parameters here mostly arrived at by trial and error.
    lines = cv2.HoughLinesP(
        im_bw, 1, np.pi / 180, 200, minLineLength=width / 12, maxLineGap=width / 150
    )

    # Collect the angles of these lines (in radians)
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angles.append(np.arctan2(y2 - y1, x2 - x1))

    # If the majority of our lines are vertical, this is probably a landscape image
    landscape = np.sum([abs(angle) > np.pi / 4 for angle in angles]) > len(angles) / 2

    # Filter the angles to remove outliers based on max_skew
    if landscape:
        angles = [
            angle
            for angle in angles
            if np.deg2rad(90 - max_skew) < abs(angle) < np.deg2rad(90 + max_skew)
        ]
    else:
        angles = [angle for angle in angles if abs(angle) < np.deg2rad(max_skew)]

    if len(angles) < 5:
        # Insufficient data to deskew
        return im

    # Average the angles to a degree offset
    angle_deg = np.rad2deg(np.median(angles))

    # If this is landscape image, rotate the entire canvas appropriately
    if landscape:
        if angle_deg < 0:
            im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
            angle_deg += 90
        elif angle_deg > 0:
            im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE)
            angle_deg -= 90

    # Rotate the image by the residual offset
    M = cv2.getRotationMatrix2D((width / 2, height / 2), angle_deg, 1)
    im = cv2.warpAffine(im, M, (width, height), borderMode=cv2.BORDER_REPLICATE)
    return im

In [177]:
def cv_process(path):
    img = cv2.imread(path)

    #Increase contrast

    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    limg = cv2.merge((cl,a,b))

    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    #Opencv denoise

    dn = cv2.fastNlMeansDenoisingColored(enhanced_img,None,15,15,7,21)
    
    cv2.imwrite('temp.jpg', dn)

    temp = cv2.imread('temp.jpg',0)

    dn = cv2.medianBlur(temp,1)
    
    dn= cv2.adaptiveThreshold(dn,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
    
    result = deskew(dn)
    
    cv2.imwrite('Clean_'+f, result)

    a = 1

    return a 

In [178]:
def findTextRegion(img):
    region = []
 
    # 1. 查找轮廓 (Detect Conour)
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    avg_area = 0
    min_area = cv2.contourArea(contours[0])
    avg_height = 0
    
    for i in range(len(contours)):
        cnt = contours[i]
        avg_area = cv2.contourArea(cnt) + avg_area
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        avg_height = avg_height + abs(box[0][1] - box[2][1])
        if cv2.contourArea(cnt) < min_area:
            min_area = cv2.contourArea(cnt)
    
    avg_area = avg_area / len(contours)
    avg_height = avg_height / len(contours)
    
    # 2. 筛选那些面积小的 (Rid off blocks with small areas,sometimes it may regard small black dots as blocks)
    for i in range(len(contours)):
        cnt = contours[i]
        # 计算该轮廓的面积 (Calculate each shade's area)
        area = cv2.contourArea(cnt) 
 
        # 面积小的都筛选掉 (Rid off blocks with small areas)
        if(area < avg_area*0.3):
            continue
 
        # 轮廓近似，作用很小 
        epsilon = 0.001 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
 
        # 找到最小的矩形，该矩形可能有方向 (Find the smallest rectangle)
        rect = cv2.minAreaRect(cnt)
        #print "rect is: "
        #print rect
 
        # box是四个点的坐标 (the box take down the four vertices of the rectangle)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
 
        # 计算高和宽 (Calculate height and width)
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
 
        # 筛选那些太细的矩形，留下扁的 (rid off rectangle that are too thin)
        if(height > width * 1.2):
            continue
        
        #筛选那些太扁的 (rid off rectangle that are too flat)
        if(height < avg_height*0.8):
            continue
 
        region.append(box)
 
    return region

In [206]:
def preprocess(gray):
    # 1. Sobel算子，x方向求梯度
    sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize = 3)
    # 2. 二值化 (Binarize)
    ret, binary = cv2.threshold(sobel, 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
 
    # 3. 膨胀和腐蚀操作的核函数 (The kernel for dilute and erosion)
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (30,9))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (30,6))
 
    # 4. 膨胀一次，让轮廓突出 (1th dilate, extract the contour of the figure)
    dilation = cv2.dilate(binary, element2, iterations = 1)
 
    # 5. 腐蚀一次，去掉细节，如表格线等。注意这里去掉的是竖直的线 (1th erosion, rid off the grid lines and details)
    erosion = cv2.erode(dilation, element1, iterations = 2)
 
    # 6. 再次膨胀，让轮廓明显一些 (2th dilation, extract the contour for each word block)
    dilation2 = cv2.dilate(erosion, element2, iterations = 3)
 
    # 7. 存储中间图片 (The images between each process)
    cv2.imwrite("binary.png", binary)
    cv2.imwrite("dilation.png", dilation)
    cv2.imwrite("erosion.png", erosion)
    cv2.imwrite("dilation2.png", dilation2)
 
    return dilation2

In [189]:
def detect(img,name):
    # 1.  转化成灰度图 （Turn into gray image）
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # 2. 形态学变换的预处理，得到可以查找矩形的图片 (Use preprocess to perform dilute and erosion)
    dilation = preprocess(gray)
 
    # 3. 查找和筛选文字区域 (Use FindTextRegion to get the word block coordinate)
    region = findTextRegion(dilation)
 
    # 4. 用绿线画出这些找到的轮廓 (Use Green lines to draw the contour)
    for box in region:
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
 
    #cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    #cv2.imshow("img", img)
 
    # 带轮廓的图片
    cv2.imwrite(name+"_"+"contours.png", img)
 
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return(region)

In [190]:
def my_ocr(img_path):
    ocr = PaddleOCR(use_angle_cls=True, lang='ch') # need to run only once to download and load model into memory
    result = ocr.ocr(img_path, det=False)
    
    for line in result:
        print(line)
        
    return result

In [182]:
def getcor(i,coordinates):
    
    x = [] ; y = []

    for temp in coordinates[i]:
        x.append(temp[1])
        y.append(temp[0])

    xmax = max(x) ; xmin = min(x)
    ymax = max(y) ; ymin = min(y)

    result = [xmin,xmax,ymin,ymax]
    
    return result

In [183]:
def crop(img,cor):
    
    result = img[cor[0]:cor[1],cor[2]:cor[3]]
    
    return result

In [184]:
def avg_distance(coordinate):
    a = 0
    for i in range (len(coordinate)-1):
        dis1 = getcor(i,coordinate)
        dis2 = getcor(i+1,coordinate)
        a = a + (dis2[0]+dis2[1] - dis1[0] - dis1[1])
    
    return (a/2)/len(coordinate)

In [185]:
def impath(img_dir,img_name):
    
    imagePath = []
    
    for i in img_name:
        imagePath.append(img_dir + '/'+ i)
    
    return(imagePath)

In [191]:
img_dir = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/OCR Dataset'
result_dir = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/Denoise_Result' 
img_name = sorted(os.listdir(img_dir))[2:]

In [192]:
os.chdir(result_dir) 
for f in img_name:
      cv_process( img_dir + '/'+ f)

In [207]:
img_dir = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/Denoise_Result' 
result_dir = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/Result' 
img_name = (sorted(os.listdir(img_dir)))[1:len(sorted(os.listdir(img_dir)))-1]
imPath = impath(img_dir,img_name)

In [208]:
#Show detection
os.chdir(result_dir) 
nameindex = 0
for imagePath in imPath:
    
    if __name__ == '__main__':
        # 读取文件
        #imagePath = sys.argv[1]
        img = cv2.imread(imagePath)
        cor=detect(img,img_name[nameindex][img_name[nameindex].index('0'):img_name[nameindex].index('.')])
        nameindex += 1
        


In [137]:
nameindex = 0
image_error = []
for imagePath in imPath:
    
    error_list = []
    
    if __name__ == '__main__':
        # 读取文件
        #imagePath = sys.argv[1]
        img = cv2.imread(imagePath)
        cor=detect(img,img_name[nameindex][img_name[nameindex].index('0'):img_name[nameindex].index('.')])

    coordinates=cor[::-1]
    corindex = 0
    #剔除坐标在图片外面的(Rid off coordinate that are outside the image)
    for coor1 in coordinates:
        for coor2 in coor1:
            for coor3 in coor2:
                if coor3 < 0:
                    del coordinates[corindex]
        
        corindex +=1
                    
    # scale down the coordinate size for easy comparison
    coordinates_int10=np.round(np.divide(coordinates,10))
    numberofrows=len(coordinates)

    columns=[['']]
    numofparalines=0
    dis = avg_distance(coordinates_int10)

    for i in range(numberofrows):

        if i >= 1:
            tempcor1_int10 = getcor(i-1,coordinates_int10)
            tempcor2_int10 = getcor(i,coordinates_int10)

        #i = 0 is the First row
        
        if i==0:
            
            #Get the coordinate of the first word block
            tempcor = getcor(i,coordinates)
            
            #Crop the word block out using the coordinat
            crop_img = crop(img,tempcor)
            
            #Perform Paddle OCR on the crop image
            result_temp = my_ocr(crop_img)[0]
            
            #Store the OCR result into list
            result = [result_temp[0]]
            
            # Store the error for the first block
            error = result_temp[1]
            
            #Store the horizontal coordinate for the word block, used for comparison between
            rowcor = []
            
            if result != ['']:
                columns.append(result)
                error_list.append(error)
                #print (error_list)
                #blockindex.append(i)
            else:
                numofparalines+=1

        #This elif is activated if the last distance is bigger than 2 * average distance, then a new row begins
        elif (tempcor2_int10[0]+tempcor2_int10[1])/2 - (tempcor1_int10[0]+tempcor1_int10[1])/2> 2*dis: # should be the same as the lower one

            rowcor = [] # Use to take down the index of each row

            tempcor = getcor(i,coordinates)
            crop_img = crop(img,tempcor)
            #result = [pytesseract.image_to_string (crop_img, lang='chi_sim')]
            result_temp = my_ocr(crop_img)[0]
            result = [result_temp[0]]
            error = result_temp[1]
            
            if result != ['']:
                columns.append(result)
                error_list.append(error)
                #print (error_list)
                startrowindex = i
                rowcor.append(tempcor[2])
            else:
                numofparalines+=1

        #This part is activated if the last distance is smaller than 2 * average distance, then remains in the same row.
        else:
            numofparalines+=1
            tempcor1 = getcor(i-1,coordinates)
            tempcor2 = getcor(i,coordinates)

            tempcor = getcor(i,coordinates)
            crop_img = crop(img,tempcor)
            result_temp = my_ocr(crop_img)[0]
            result = result_temp[0]
            error = result_temp[1]

            if result != ['']:
                columns[i-numofparalines].append(result)
                rowcor.append(tempcor[2])
                error_list.append(error)
                #print (error_list)

            if i <= numberofrows-2:
                tempcor3_int10 = getcor(i+1,coordinates_int10)

                if (tempcor3_int10[0]+tempcor3_int10[1])/2 - (tempcor2_int10[0]+tempcor2_int10[1])/2> 2*dis: # should be the same as in the previous one

                    if len(rowcor) != 0:
                        
                        if len(columns)==1:
                            sortedrowcor,sortedrow = zip(*sorted(zip(rowcor,columns[-1])))
                            del columns[-1]
                            columns.append(list(sortedrow))
                            
                        else:
                            sortedrowcor,sortedrow = zip(*sorted(zip(rowcor,columns[-2])))

#                             print('cloumns[-2]')
#                             print(columns[-2])
#                             print('rowcor:')
#                             print(rowcor)
#                             print('sortedrowcor:')
#                             print(sortedrowcor)
#                             print('sortedrow:')
#                             print(sortedrow)
                            tempcolumn = columns[-1]
                            del columns[-1]
                            del columns[-1]
                            columns.append(list(sortedrow))
                            columns.append(tempcolumn)


    df=pd.DataFrame(columns)
    df.fillna('',inplace=True)

    # path="result/result.xlsx"
    # if not os.path.exists(path):
    #     os.makedirs(path)
    
    name = img_name[nameindex]
    stop = name.index('.')
    df.to_excel(name[0:stop]+'_result.xlsx',index=False)
    nameindex = nameindex + 1
    image_error.append(sum(error_list)/len(error_list))
#The error for each image
print(image_error)
#The error for 4 image
print(sum(image_error)/4)

[2022/07/24 19:36:40] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_p

('序号：000', 0.8152449131011963)
[2022/07/24 19:36:42] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='

('本次价231360婴易码质143米', 0.5581960678100586)
[2022/07/24 19:36:44] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_l

('图', 0.19277875125408173)
[2022/07/24 19:36:46] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max'

('509999', 0.9849410057067871)
[2022/07/24 19:36:48] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='

...

('￥519.96', 0.954041063785553)
[2022/07/24 19:40:32] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='

('价税合计（大写）', 0.8977357149124146)
[2022/07/24 19:40:34] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type

('纳税人识别号：91520621MA6E5H3C18', 0.9793033599853516)
[2022/07/24 19:40:36] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=76

('开户行及账号：中国建设银行江口县支行 52001686336052501242', 0.9448283314704895)
[2022/07/24 19:40:38] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_lim

('收款人：铜仁供电', 0.9685794711112976)
[2022/07/24 19:40:39] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type